In [1]:
# Для выгрузки данных с ВК
import requests
import json
import os
from auth_data import token # сервисный токен
from auth_data import tokenbgd # личный токен

# Для обработки полученных данных
import pandas as pd
import numpy as np

# <center> Получаем данные постов группы

In [2]:
# Запрашиваемые параметры
group_name = 'atheism' # Название группы взятое из ссылки
count_last_posts = 40 # Число последних постов

# Производим запрос
url_group = f"https://api.vk.com/method/wall.get?domain={group_name}&count={count_last_posts}&access_token={token}&v=5.81"
req = requests.get(url_group)
group_json = req.json()

# Создаем список с id номерами постов группы
post_id_list = list()
for i in range(len(group_json['response']['items'])):
    post_id_list.append(group_json['response']['items'][i]['id'])

# Проверяем существует ли директория с именем группы
if os.path.exists(f"{group_name}"):
    print(f"Директория с именем {group_name} уже существует!")
else:
    os.mkdir(group_name)
    # сохраняем данные в json файл, чтобы видеть структуру
    
with open(f"{group_name}/{group_name}.json", "w", encoding="utf-8") as file:
    json.dump(group_json, file, indent=4, ensure_ascii=False)

print("id постов:",post_id_list)

Директория с именем atheism уже существует!
id постов: [2794407, 2794126, 2793803, 2793651, 2793616, 2793544, 2793477, 2793427, 2793321, 2793191, 2793003, 2792911, 2792632, 2792491, 2792311, 2792101, 2792039, 2791899, 2791772, 2791621, 2791384, 2791265, 2791185, 2791046, 2790974, 2790902, 2790861, 2790760, 2790678, 2790636, 2790522, 2790355, 2790229, 2790129, 2790016, 2789793, 2789663, 2789515, 2789339, 2789278]


# <center> Получение комментариев

In [3]:
# Запрашиваемые параметры
owner_id = group_json['response']['items'][0]['from_id'] #'atheism' # Название группы взятое из ссылки

# Создаем списки с пользователями и текстом комментариев
text = list()
post_id_json = list()

for j in range(len(post_id_list)):
    # Производим запрос
    url_posts = f"https://api.vk.com/method/wall.getComments?owner_id={owner_id}&post_id={post_id_list[j]}&count=100&access_token={token}&v=5.81"
    req = requests.get(url_posts)
    post_id_json.append(req.json())
    for i in range(len(post_id_json[j]['response']['items'])):
        text.append([
            owner_id,                                               # id группы
            post_id_list[j],                                        # id поста
            post_id_json[j]['response']['items'][i]['from_id'],     # id пользователя 
            post_id_json[j]['response']['items'][i]['text']],       # текст комментария            
        )

# Проверяем существует ли директория с именем группы
if os.path.exists(f"{group_name}/{owner_id}/{owner_id}_text")==False:
    os.makedirs(f"{group_name}/{owner_id}/{owner_id}_text")

# Сохраняем данные в json файл, чтобы видеть структуру       
with open(f"{group_name}/{owner_id}/{owner_id}_text/{owner_id}_text.json", "w", encoding="utf-8") as file:
    json.dump(text, file, indent=4, ensure_ascii=False)


# <center> Создаем таблицу, куда будем вносить данные о пользователях

In [4]:
users_data = pd.DataFrame(
    data = text,
    columns= ['id_группы', 'id_поста', 'id_пользователя', 'Комментарий']
)

In [5]:
users_data.head()

,id_группы,id_поста,id_пользователя,Комментарий
0,-63683472,2794407,655386741,Правильно. За наших парней из Самары.
1,-63683472,2794407,588757522,"Ебучие пригожинские выродки, фас!!!"
2,-63683472,2794407,621623505,А где снег? Какие - то старые фото 🤣👍
3,-63683472,2794407,540038594,Мрази..
4,-63683472,2794407,339042592,Про Днепр забыли умышленно?


In [6]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2679 entries, 0 to 2678
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_группы        2679 non-null   int64 
 1   id_поста         2679 non-null   int64 
 2   id_пользователя  2679 non-null   int64 
 3   Комментарий      2679 non-null   object
dtypes: int64(3), object(1)
memory usage: 83.8+ KB


# <center> Добавление данных о городе пользователя

In [7]:
# Напишем функцию, возвращающую имя пользователя
def user_info(user_id, token=token): 
    # Производим запрос
    url_posts = f"https://api.vk.com/method/users.get?user_ids={user_id}&fields=city&access_token={token}&v=5.81"
    req = requests.get(url_posts)
    users_info_json = req.json()    

    try:
        first_name = users_info_json['response'][0]['first_name']
        last_name = users_info_json['response'][0]['last_name']
        city = users_info_json['response'][0]['city']['title']         
    except KeyError:
        city = 0
    except IndexError:
        return 0, 0, 0  

    return first_name, last_name, city

In [8]:
users_data['Инфо'] = users_data['id_пользователя'].apply(user_info)

In [9]:
users_data['Имя'] = users_data['Инфо'].apply(lambda x: x[0])
users_data['Фамилия'] = users_data['Инфо'].apply(lambda x: x[1])
users_data['Город'] = users_data['Инфо'].apply(lambda x: x[2])

users_data = users_data.drop(['Инфо'], axis=1) 

In [10]:
users_data.head(10)

,id_группы,id_поста,id_пользователя,Комментарий,Имя,Фамилия,Город
0,-63683472,2794407,655386741,Правильно. За наших парней из Самары.,Kvantovy,Skachok,0
1,-63683472,2794407,588757522,"Ебучие пригожинские выродки, фас!!!",Syd,Barrett,0
2,-63683472,2794407,621623505,А где снег? Какие - то старые фото 🤣👍,Alexander,Gelman,0
3,-63683472,2794407,540038594,Мрази..,Elena,Abramova,Saint Petersburg
4,-63683472,2794407,339042592,Про Днепр забыли умышленно?,Evgeny,Chechetkin,0
5,-63683472,2794407,66196026,То я смотрю комментариев мало. Не старлинки ну...,Pavel,Kravchenko,Lipetsk
6,-63683472,2794407,204208359,"[id655386741|Квантовый], а что,украинцы обстре...",Vyacheslav,Streltsov,Elets
7,-63683472,2794407,316589068,"[id655386741|Квантовый], новых парней себе най...",Val,Gri,0
8,-63683472,2794407,124294782,,Yura,Buntov,Vyborg
9,-63683472,2794407,456664204,"Горели б вы в аду, орки!",Robert,Ross,0


In [11]:
# Генерируем ссылку на пост с комментариями
def url_to_post(id_group, id_post):
    return f"https://vk.com/atheism?w=wall{id_group}_{id_post}"

users_data['Ссылка_на_пост'] = 0
for i in range(users_data.shape[0]):
    id_group = users_data['id_группы'].iloc[i]
    id_post = users_data['id_поста'].iloc[i]
    users_data['Ссылка_на_пост'].iloc[i] = url_to_post(id_group, id_post)

c:\Users\mish9\.conda\envs\sf\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [12]:
# Генерируем ссылку на аккаунт пользователя
def url_to_user(id_user):
    return f"https://vk.com/id{id_user}"

users_data['Ссылка_на_пользователя'] = users_data['id_пользователя'].apply(url_to_user)

In [13]:
users_data

,id_группы,id_поста,id_пользователя,Комментарий,Имя,Фамилия,Город,Ссылка_на_пост,Ссылка_на_пользователя
0,-63683472,2794407,655386741,Правильно. За наших парней из Самары.,Kvantovy,Skachok,0,https://vk.com/atheism?w=wall-63683472_2794407,https://vk.com/id655386741
1,-63683472,2794407,588757522,"Ебучие пригожинские выродки, фас!!!",Syd,Barrett,0,https://vk.com/atheism?w=wall-63683472_2794407,https://vk.com/id588757522
2,-63683472,2794407,621623505,А где снег? Какие - то старые фото 🤣👍,Alexander,Gelman,0,https://vk.com/atheism?w=wall-63683472_2794407,https://vk.com/id621623505
3,-63683472,2794407,540038594,Мрази..,Elena,Abramova,Saint Petersburg,https://vk.com/atheism?w=wall-63683472_2794407,https://vk.com/id540038594
4,-63683472,2794407,339042592,Про Днепр забыли умышленно?,Evgeny,Chechetkin,0,https://vk.com/atheism?w=wall-63683472_2794407,https://vk.com/id339042592
...,...,...,...,...,...,...,...,...,...
2674,-63683472,2789278,505595623,"[id497915440|Остап], у неё просто мозг отрофир...",Pain,Bastard,Moscow,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id505595623
2675,-63683472,2789278,436193921,"[id175386642|Мария], им он не нужен , призыв н...",Kirill,Chumarov,Saint Petersburg,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id436193921
2676,-63683472,2789278,436193921,"[id175386642|Мария], как и граждан США это кас...",Kirill,Chumarov,Saint Petersburg,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id436193921
2677,-63683472,2789278,658250361,"[id140573223|Владимир], слышал даже могут приз...",Jewgienij,Wolnow,Vancouver,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id658250361


# <center> Отфильтровываем данные с Иркутском

In [21]:
Irkutsk = users_data[users_data['Город']=='Irkutsk']
Irkutsk

,id_группы,id_поста,id_пользователя,Комментарий,Имя,Фамилия,Город,Ссылка_на_пост,Ссылка_на_пользователя
25,-63683472,2794126,33196285,"Видите, там даже бомжи более культурные и прия...",Lyonya,Golubkov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2794126,https://vk.com/id33196285
1614,-63683472,2791046,342957915,А ладу дадут?😆,Ksenia,Petrovna,Irkutsk,https://vk.com/atheism?w=wall-63683472_2791046,https://vk.com/id342957915
1793,-63683472,2790974,342957915,"[id34457849|Михаил], это цивилизованные методы...",Ksenia,Petrovna,Irkutsk,https://vk.com/atheism?w=wall-63683472_2790974,https://vk.com/id342957915
2075,-63683472,2790636,171716341,"Ну педераст, товарищи, пе-де-раст... Что нового?)",Boris,Pavlov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2790636,https://vk.com/id171716341
2099,-63683472,2790229,342957915,"Главное не гейевропеец, а то скрепы да и сами ...",Ksenia,Petrovna,Irkutsk,https://vk.com/atheism?w=wall-63683472_2790229,https://vk.com/id342957915
2585,-63683472,2789278,12960639,Интересно а как будут служить 30 летние с 18 л...,Vovchik,Zakharov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id12960639
2616,-63683472,2789278,12960639,"[id669233873|Ханк], научи как читать жопой?",Vovchik,Zakharov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id12960639
2617,-63683472,2789278,12960639,Согласен,Vovchik,Zakharov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id12960639


In [15]:
users_data.to_csv('users_data.csv', sep=';', decimal='.', index=True)